In [1]:
!pip install nystrom-attention
!pip install pytorch_toolbelt

  Obtaining dependency information for nystrom-attention from https://files.pythonhosted.org/packages/98/ae/0e5579633aea9cbf9b5c0abc0a18dd81d1133fc0db58797830269856efcd/nystrom_attention-0.0.12-py3-none-any.whl.metadata
  Obtaining dependency information for einops>=0.7.0 from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00
  Obtaining dependency information for pytorch_toolbelt from https://files.pythonhosted.org/packages/5c/73/1c9f912e25491bceb63e5ce2d88c612069d4a761373fb3f1220d4724cacc/pytorch_toolbelt-0.6.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 3.4 MB/s eta 0:00:00


In [2]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e14

[I 2024-03-23 15:12:09.012 ServerApp] Extension package beatrix_jupyterlab took 5.6222s to import
[W 2024-03-23 15:12:09.052 ServerApp] A `_jupyter_server_extension_points` function was not found in jupyter_lsp. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyter Server.
[I 2024-03-23 15:12:09.217 ServerApp] Extension package jupyter_server_proxy took 0.1602s to import
[I 2024-03-23 15:12:10.100 ServerApp] Extension package jupytext took 0.3283s to import
[W 2024-03-23 15:12:10.109 ServerApp] A `_jupyter_server_extension_points` function was not found in nbclassic. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyter Server.
[W 2024-03-23 15:12:10.110 ServerApp] A `_jupyter_server_extension_points` function was not found in notebook_shim. Instead, a `_jupyter_server_extension_pa

In [3]:
#----> path
from pathlib import Path
import glob
import os

#----> utils
import sys
import numpy as np
import inspect # 查看python 类的参数和模块、函数代码
import importlib # In order to dynamically import the library
import random
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

#----> pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from nystrom_attention import NystromAttention
from torch.utils.data import random_split, DataLoader, Dataset
from torchvision.datasets import MNIST
from torchvision import transforms

# Loss
from pytorch_toolbelt import losses as L

# Optim
import math
import torch.optim as optim
from torch.optim.optimizer import Optimizer
from collections import defaultdict
try:
    from apex.optimizers import FusedNovoGrad, FusedAdam, FusedLAMB, FusedSGD
    has_apex = True
except ImportError:
    has_apex = False


#----> pytorch_lightning
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import loggers as pl_loggers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
CONFIG = {
    # Config
    "config": 'UBC',
    
    # Generate
    "seed": 42,
    "devices": 1,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "benchmark": True,
    "precision": 32,
    "epochs": 100,
    "grad_acc": 2,
    "patience": 10,
    "server": "train", #train #test
    "log_path": "logs/",
    "save_top_k": 1,
    "test_size": 0.2,
    
    # Data
    "dataset_name": "UBC_data",
    "data_shuffle": True,
    "data_dir": "/kaggle/input/feature-scale0-25/feature-0.25scale/pt_files",
    "dataset_dir": "/kaggle/working/dataset_csv/UBC16",
    "fold": 0,
    "nfold": 4,
    
    "train_dataloader": {
           "batch_size": 1, 
            "num_workers": 8,
       },

    "test_dataloader": {
            "batch_size": 1,
            "num_workers": 8,
        },
    
    # Model
    "name": "TransMIL",
    "n_classes": 5,
    
    # Optimizer
    "opt": "lookahead_radam",
    "lr": 0.0002,
    "opt_eps": None, 
    "opt_betas": None,
    "momentum": None, 
    "weight_decay": 0.00001,
    
    # Loss
    "base_loss": "focal",
}

label_encoder = {
    "MC": 0,
    "EC": 1,
    "CC": 2,
    "HGSC": 3,
    "LGSC": 4,
}

label_decoder = {v:k for k,v in label_encoder.items()}

def transform_data(s):
    s['image_id'] = str(s['image_id'])
    s['label'] = label_encoder[s['label']]
    return s

In [5]:
def drop_useless_img(df_data):
    useless_list = [281, 3222, 5264, 9154, 12244, 26124, 31793, 32192, 33839, 41099, 52308, 54506, 63836, 1289, 32035]
    df_data = df_data[ ~df_data['image_id'].isin(useless_list) ]
    if 15583 in df_data['image_id']:
        df_data[ df_data['image_id'] == 15583 ]['label'] = 'MC'
    return df_data


def generate_dataset_csv(use_tma = True):        
    
    df_data = pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv')
    df_data = drop_useless_img(df_data)
    df_data = df_data.apply(lambda s: transform_data(s), axis=1)
    
    slide_ids = os.listdir(CONFIG['data_dir'])
    slide_ids = [os.path.splitext(slide_id)[0] for slide_id in slide_ids]
    cnt = 0
    for slide_id in slide_ids:
        if slide_id in df_data['image_id'].values:
            continue
        else:
            cnt += 1
            print(f"slide id: {slide_id} is remove.")
    
    df_data = df_data[ df_data['image_id'].isin(slide_ids) ]
    print(f"df_data len is {len(df_data)}, slide_ids len is {len(slide_ids)}, drop ids num is {cnt}.")
    
    df_tma = df_data[ df_data['is_tma'] == True ]
    df_data = df_data[ ~df_data['image_id'].isin(df_tma['image_id'].values) ].loc[:, ['image_id', 'label']].reset_index(drop=True)
    
    print(f"df_tma len is {len(df_tma)}, df_data len is {len(df_data)}.")
    
    if CONFIG['nfold'] >= 2:
        # 分层交叉验证
        skf = StratifiedShuffleSplit(n_splits=CONFIG['nfold'], test_size=CONFIG['test_size'])
        for fold, (train_idxs, val_idxs) in enumerate(skf.split(df_data['image_id'], df_data['label'])):
            print("分层随机划分：%s %s" % (train_idxs.shape, val_idxs.shape))
            # 训练集
            df_train = df_data.iloc[train_idxs, :].reset_index(drop=True)
            # 验证集
            df_val = df_data.iloc[val_idxs, :].reset_index(drop=True)
            # 测试集
            df_test = df_tma.loc[:, ['image_id', 'label']].sample(frac=1.0).reset_index(drop=True)
            # 重新命名列名
            columns_name = ["train_id", "train_label", "val_id", "val_label", "test_id", "test_label"]
            df_dataset = pd.concat([df_train, df_val, df_test], axis=1, ignore_index=True).set_axis(columns_name, axis=1)
            # 创建 dataset_csv 的存储路径
            Path(CONFIG['dataset_dir']).mkdir(exist_ok=True, parents=True)
            df_dataset.to_csv(os.path.join(CONFIG['dataset_dir'], f"fold{fold}.csv"), index=False)
            display(df_dataset)
    else:
        raise Exception('nfold should be bigger than 2.')
    
generate_dataset_csv(use_tma = True)

df_data len is 523, slide_ids len is 523, drop ids num is 0.
df_tma len is 25, df_data len is 498.
分层随机划分：(398,) (100,)


,train_id,train_label,val_id,val_label,test_id,test_label
0,18914,3,43998,1.0,41586,2.0
1,43875,3,52931,2.0,50932,3.0
2,23796,4,50304,1.0,42857,2.0
3,50962,1,34688,4.0,47035,1.0
4,46139,3,39252,0.0,40864,4.0
...,...,...,...,...,...,...
393,55281,3,NaN,NaN,NaN,NaN
394,59515,2,NaN,NaN,NaN,NaN
395,22155,2,NaN,NaN,NaN,NaN
396,44232,1,NaN,NaN,NaN,NaN


分层随机划分：(398,) (100,)


,train_id,train_label,val_id,val_label,test_id,test_label
0,61089,3,54928,2.0,61797,3.0
1,50712,3,56861,2.0,44603,2.0
2,62641,3,27245,3.0,36302,2.0
3,30539,3,26603,3.0,17637,3.0
4,21260,4,6449,2.0,42857,2.0
...,...,...,...,...,...,...
393,57711,1,NaN,NaN,NaN,NaN
394,25561,3,NaN,NaN,NaN,NaN
395,52420,2,NaN,NaN,NaN,NaN
396,57265,0,NaN,NaN,NaN,NaN


分层随机划分：(398,) (100,)


,train_id,train_label,val_id,val_label,test_id,test_label
0,56861,2,2906,1.0,50932,3.0
1,7204,1,47105,1.0,13568,4.0
2,65371,3,18813,2.0,44603,2.0
3,30738,3,30868,0.0,47035,1.0
4,18547,3,57711,1.0,42857,2.0
...,...,...,...,...,...,...
393,34688,4,NaN,NaN,NaN,NaN
394,6281,1,NaN,NaN,NaN,NaN
395,22740,1,NaN,NaN,NaN,NaN
396,63429,1,NaN,NaN,NaN,NaN


分层随机划分：(398,) (100,)


,train_id,train_label,val_id,val_label,test_id,test_label
0,27747,3,18914,3.0,35565,0.0
1,52846,1,56799,0.0,53655,1.0
2,64629,3,53402,1.0,36302,2.0
3,51893,0,40888,2.0,36583,4.0
4,35792,0,50048,1.0,17637,3.0
...,...,...,...,...,...,...
393,43815,3,NaN,NaN,NaN,NaN
394,26950,2,NaN,NaN,NaN,NaN
395,970,3,NaN,NaN,NaN,NaN
396,44976,2,NaN,NaN,NaN,NaN


In [6]:
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt(
                            (1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (
                                        N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0 and group['weight_decay'] is not None:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

In [7]:
def load_loggers():

    log_path = CONFIG['log_path']
    Path(log_path).mkdir(exist_ok=True, parents=True)
    
    log_name = CONFIG['config'] 
    version_name = CONFIG['name']
    CONFIG['fold_log_path'] = Path(log_path) / log_name / version_name / f"fold{CONFIG['fold']}"  # CONFIG['fold'] 在交叉验证过程中不断改变
    print(f"---->Log dir: {CONFIG['fold_log_path']}")
    
    #---->TensorBoard
    tb_logger = pl_loggers.TensorBoardLogger(log_path+str(log_name),
                                             name = version_name, version = f"fold{CONFIG['fold']}",
                                             log_graph = True, default_hp_metric = False)
    #---->CSV
    csv_logger = pl_loggers.CSVLogger(log_path+str(log_name),
                                      name = version_name, version = f"fold{CONFIG['fold']}", )
    
    return [tb_logger, csv_logger]


def load_callbacks():

    Mycallbacks = []
    # Make output path
    output_path = CONFIG['fold_log_path']
    output_path.mkdir(exist_ok=True, parents=True)

    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.00,
        patience=CONFIG['patience'],
        verbose=True,
        mode='min'
    )
    Mycallbacks.append(early_stop_callback)

    if CONFIG['server'] == 'train':
        Mycallbacks.append(ModelCheckpoint(monitor = 'val_loss',
                                         dirpath = str(CONFIG['fold_log_path']),
                                         filename = '{epoch:02d}-{val_loss:.4f}',
                                         verbose = True,
                                         save_last = True,
                                         save_top_k = CONFIG['save_top_k'],
                                         mode = 'min',
                                         save_weights_only = True))
    return Mycallbacks


def cross_entropy_torch(x, y):
    x_softmax = [F.softmax(x[i]) for i in range(len(x))]
    x_log = torch.tensor([torch.log(x_softmax[i][y[i]]) for i in range(len(y))])
    loss = - torch.sum(x_log) / len(y)
    return loss


def create_loss(select_loss, w1=1.0, w2=0.5):
    conf_loss = select_loss
    ### MulticlassJaccardLoss(classes=np.arange(11)
    # mode = args.base_loss #BINARY_MODE \MULTICLASS_MODE \MULTILABEL_MODE 
    loss = None
    if hasattr(nn, conf_loss): 
        loss = getattr(nn, conf_loss)() 
    #binary loss
    elif conf_loss == "focal":
        loss = L.CrossEntropyFocalLoss()
    elif conf_loss == "jaccard":
        loss = L.BinaryJaccardLoss()
    elif conf_loss == "jaccard_log":
        loss = L.BinaryJaccardLoss()
    elif conf_loss == "dice":
        loss = L.BinaryDiceLoss()
    elif conf_loss == "dice_log":
        loss = L.BinaryDiceLogLoss()
    elif conf_loss == "dice_log":
        loss = L.BinaryDiceLogLoss()
    elif conf_loss == "bce+lovasz":
        loss = L.JointLoss(BCEWithLogitsLoss(), L.BinaryLovaszLoss(), w1, w2)
    elif conf_loss == "lovasz":
        loss = L.BinaryLovaszLoss()
    elif conf_loss == "bce+jaccard":
        loss = L.JointLoss(BCEWithLogitsLoss(), L.BinaryJaccardLoss(), w1, w2)
    elif conf_loss == "bce+log_jaccard":
        loss = L.JointLoss(BCEWithLogitsLoss(), L.BinaryJaccardLogLoss(), w1, w2)
    elif conf_loss == "bce+log_dice":
        loss = L.JointLoss(BCEWithLogitsLoss(), L.BinaryDiceLogLoss(), w1, w2)
    elif conf_loss == "reduced_focal":
        loss = L.BinaryFocalLoss(reduced=True)
    else:
        assert False and "Invalid loss"
        raise ValueError
    return loss


In [8]:

class UBCData(Dataset):
    def __init__(self, state=None):
        # Set all input args as attributes
        self.__dict__.update(locals())

        #---->data and label
        self.nfolds = CONFIG['nfold']
        self.fold = CONFIG['fold']
        self.feature_dir = CONFIG['data_dir']
        self.csv_dir = os.path.join(CONFIG['dataset_dir'], f'fold{self.fold}.csv')
        self.slide_data = pd.read_csv(self.csv_dir)

        #---->order
        self.shuffle = CONFIG['data_shuffle']

        #---->split dataset
        if state == 'train':
            self.data = self.slide_data.loc[:, 'train_id'].dropna()
            self.label = self.slide_data.loc[:, 'train_label'].dropna()
        if state == 'val':
            self.data = self.slide_data.loc[:, 'val_id'].dropna()
            self.label = self.slide_data.loc[:, 'val_label'].dropna()
        if state == 'test':
            self.data = self.slide_data.loc[:, 'test_id'].dropna()
            self.label = self.slide_data.loc[:, 'test_label'].dropna()


    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, idx):
        slide_id = int(self.data[idx])
        label = int(self.label[idx]) # 数据中存在 NAN 会自动转化为浮点型，因此转化成整数
        pt_path = os.path.join(CONFIG['data_dir'], str(slide_id) + '.pt')
#         print(pt_path)
        features = torch.load(pt_path)

        #----> shuffle
        if self.shuffle == True:
            index = [x for x in range(features.shape[0])]
            random.shuffle(index)
            features = features[index]


        return features, label

    
# my_dataset = UBCData(state='train')
# for i in range(5):
#     feature, label = my_dataset[i]
#     print(feature.shape, label)

In [9]:
class DataInterface(pl.LightningDataModule):

    def __init__(self, train_batch_size=64, train_num_workers=8, test_batch_size=1, test_num_workers=1,dataset_name=None, **kwargs):
        """[summary]

        Args:
            batch_size (int, optional): [description]. Defaults to 64.
            num_workers (int, optional): [description]. Defaults to 8.
            dataset_name (str, optional): [description]. Defaults to ''.
        """        
        super().__init__()

        self.train_batch_size = train_batch_size
        self.train_num_workers = train_num_workers
        self.test_batch_size = test_batch_size
        self.test_num_workers = test_num_workers
        self.dataset_name = dataset_name
        self.kwargs = kwargs
        self.load_data_module()

 
    def prepare_data(self):
        pass

    
    def setup(self, stage=None):
        # 2. how to split, argument
        """  
        - count number of classes

        - build vocabulary

        - perform train/val/test splits

        - apply transforms (defined explicitly in your datamodule or assigned in init)
        """
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            self.train_dataset = self.instancialize(state='train')
            self.val_dataset = self.instancialize(state='val')
 

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.test_dataset = self.instancialize(state='test')


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.train_batch_size, num_workers=self.train_num_workers, shuffle=True)

    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.train_batch_size, num_workers=self.train_num_workers, shuffle=False)

    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.test_batch_size, num_workers=self.test_num_workers, shuffle=False)
    
    
    def load_data_module(self):
        camel_name =  ''.join([i.capitalize() for i in (self.dataset_name).split('_')])
        try:
            self.data_module = UBCData
        except:
            raise ValueError(
                'Invalid Dataset File Name or Invalid Class Name!')
    
    
    def instancialize(self, **other_args):
        """ Instancialize a model using the corresponding parameters
            from self.hparams dictionary. You can also input any args
            to overwrite the corresponding value in self.kwargs.
        """
        class_args = inspect.getargspec(self.data_module.__init__).args[1:]
        inkeys = self.kwargs.keys()
        args1 = {}
        for arg in class_args:
            if arg in inkeys:
                args1[arg] = self.kwargs[arg]
        args1.update(other_args)
        return self.data_module(**args1)

    
DataInterface_dict = {
                        'train_batch_size': CONFIG['train_dataloader']['batch_size'],
                        'train_num_workers': CONFIG['train_dataloader']['num_workers'],
                        'test_batch_size': CONFIG['test_dataloader']['batch_size'],
                        'test_num_workers': CONFIG['test_dataloader']['num_workers'],
                        'dataset_name': CONFIG['dataset_name'],
                    }
my_dm = DataInterface(**DataInterface_dict)
my_dm.setup(stage='fit')
print(len(my_dm.val_dataloader()))
for feature, label in my_dm.val_dataloader():
    print(feature.shape, label)
    break

100
torch.Size([1, 688, 384]) tensor([1])


In [10]:
class TransLayer(nn.Module):

    def __init__(self, norm_layer=nn.LayerNorm, dim=512):
        super().__init__()
        self.norm = norm_layer(dim)
        self.attn = NystromAttention(
            dim = dim,
            dim_head = dim//8,
            heads = 8,
            num_landmarks = dim//2,    # number of landmarks
            pinv_iterations = 6,    # number of moore-penrose iterations for approximating pinverse. 6 was recommended by the paper
            residual = True,         # whether to do an extra residual with the value or not. supposedly faster convergence if turned on
            dropout=0.1
        )

    def forward(self, x):
        x = x + self.attn(self.norm(x))
        x = x + self.attn(self.norm(x))
        x = x + self.attn(self.norm(x))
            
        return x


class PPEG(nn.Module):
    def __init__(self, dim=512):
        super(PPEG, self).__init__()
        self.proj = nn.Conv2d(dim, dim, 7, 1, 7//2, groups=dim)
        self.proj1 = nn.Conv2d(dim, dim, 5, 1, 5//2, groups=dim)
        self.proj2 = nn.Conv2d(dim, dim, 3, 1, 3//2, groups=dim)
        self.conv1 = nn.Conv2d(dim, dim, 1, 1, 1//2, groups=dim)
        
    def forward(self, x, H, W):
        B, _, C = x.shape
        cls_token, feat_token = x[:, 0], x[:, 1:]
        cnn_feat = feat_token.transpose(1, 2).view(B, C, H, W)
        c = self.conv1(cnn_feat)
        x = self.proj(c) + cnn_feat + self.proj1(c) + self.proj2(c) 
        x = x.flatten(2).transpose(1, 2)
        x = torch.cat((cls_token.unsqueeze(1), x), dim=1)
        
        return x


# class TransMIL(nn.Module):
#     def __init__(self, n_classes):
#         super(TransMIL, self).__init__()
#         self.pos_layer = PPEG(dim=512)
#         self._fc1 = nn.Sequential(nn.Linear(384, 512), nn.ReLU()) # 将输入特征进行升维
#         self.cls_token = nn.Parameter(torch.randn(1, 1, 512))
#         self.n_classes = n_classes
#         self.layer1 = TransLayer(dim=512)
#         self.layer2 = TransLayer(dim=512)
#         self.norm = nn.LayerNorm(512)
#         self._fc2 = nn.Linear(512, self.n_classes)


#     def forward(self, **kwargs):
#         # 输入特征 H
#         h = kwargs['data'].float() #[B, n, 384]
        
#         h = self._fc1(h) #[B, n, 512]
        
#         #---->pad
#         H = h.shape[1]
#         _H, _W = int(np.ceil(np.sqrt(H))), int(np.ceil(np.sqrt(H)))
#         add_length = _H * _W - H
#         h = torch.cat([h, h[:,:add_length,:]],dim = 1) #[B, N, 512]

#         #---->cls_token
#         B = h.shape[0]
#         cls_tokens = self.cls_token.expand(B, -1, -1).cuda()
#         h = torch.cat((cls_tokens, h), dim=1)

#         #---->Translayer x1
#         h = self.layer1(h) #[B, N, 512]

#         #---->PPEG
#         h = self.pos_layer(h, _H, _W) #[B, N, 512]
        
#         #---->Translayer x2
#         h = self.layer2(h) #[B, N, 512]

#         #---->cls_token
#         h = self.norm(h)[:,0]

#         #---->predict
#         logits = self._fc2(h) #[B, n_classes] 从这里这届预测出每个包在 n_classes 个类别上的特征映射结果
#         Y_hat = torch.argmax(logits, dim=1) # 选择概率最大的类的下标index作为包的类别
#         Y_prob = F.softmax(logits, dim = 1) # 从第1个维度开始，对每一个包的特征映射进行归一化
#         results_dict = {'logits': logits, 'Y_prob': Y_prob, 'Y_hat': Y_hat} 
        
#         # 返回三个结果，特征映射的结果，归一化的概率，预测出的包的类别，通过标签训练模型
#         return results_dict

In [11]:
import torch.nn as nn

class SelfAttentionLayer(nn.Module):
    def __init__(self, dim):
        super(SelfAttentionLayer, self).__init__()
        self.query = nn.Linear(dim, dim)
        self.key = nn.Linear(dim, dim)
        self.value = nn.Linear(dim, dim)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        
        attn_weights = torch.matmul(q, k.transpose(-2, -1)) / (x.shape[-1] ** 0.5)
        attn_weights = self.softmax(attn_weights)
        attended_values = torch.matmul(attn_weights, v)
        
        return attended_values

class TransMIL(nn.Module):
    def __init__(self, n_classes):
        super(TransMIL, self).__init__()
        self.pos_layer = PPEG(dim=512)
        self._fc1 = nn.Sequential(nn.Linear(384, 512), nn.ReLU()) # 将输入特征进行升维
        self.cls_token = nn.Parameter(torch.randn(1, 1, 512))
        self.n_classes = n_classes
        self.layer1 = TransLayer(dim=512)
        self.layer2 = TransLayer(dim=512)
        self.self_attention = SelfAttentionLayer(dim=512)  # Add Self-Attention Layer
        self.norm = nn.LayerNorm(512)
        self._fc2 = nn.Linear(512, self.n_classes)


    def forward(self, **kwargs):
        # 输入特征 H
        h = kwargs['data'].float() #[B, n, 384]
        
        h = self._fc1(h) #[B, n, 512]
        
        #---->pad
        H = h.shape[1]
        _H, _W = int(np.ceil(np.sqrt(H))), int(np.ceil(np.sqrt(H)))
        add_length = _H * _W - H
        h = torch.cat([h, h[:,:add_length,:]],dim = 1) #[B, N, 512]

        #---->cls_token
        B = h.shape[0]
        cls_tokens = self.cls_token.expand(B, -1, -1).cuda()
        h = torch.cat((cls_tokens, h), dim=1)

        #---->Translayer x1
        h = self.layer1(h) #[B, N, 512]

        #---->PPEG
        h = self.pos_layer(h, _H, _W) #[B, N, 512]
        
        #---->Self-Attention
#         h = self.self_attention(h)  # Apply Self-Attention
        
        #---->Translayer x2
        h = self.layer2(h) #[B, N, 512]

        #---->cls_token
        h = self.norm(h)[:,0]

        #---->predict
        logits = self._fc2(h) #[B, n_classes] 从这里这届预测出每个包在 n_classes 个类别上的特征映射结果
        Y_hat = torch.argmax(logits, dim=1) # 选择概率最大的类的下标index作为包的类别
        Y_prob = F.softmax(logits, dim = 1) # 从第1个维度开始，对每一个包的特征映射进行归一化
        results_dict = {'logits': logits, 'Y_prob': Y_prob, 'Y_hat': Y_hat} 
        
        # 返回三个结果，特征映射的结果，归一化的概率，预测出的包的类别，通过标签训练模型
        return results_dict

In [12]:
import sklearn.metrics

class ModelInterface(pl.LightningModule):

    #---->init
    def __init__(self, model):
        super(ModelInterface, self).__init__()
        self.save_hyperparameters()
        self.load_model()
        self.loss = create_loss(CONFIG['base_loss'])
        self.n_classes = CONFIG['n_classes']
        self.log_path = CONFIG['log_path']

        #---->acc
        self.data = [{"count": 0, "correct": 0} for i in range(self.n_classes)]
        
        #---->Metrics
        if self.n_classes > 2: 
            self.AUROC = torchmetrics.AUROC(num_classes = self.n_classes, average = 'macro', task='multiclass')
            metrics = torchmetrics.MetricCollection([torchmetrics.Accuracy(num_classes = self.n_classes,
                                                                           average='micro', task='multiclass'),
                                                     torchmetrics.CohenKappa(num_classes = self.n_classes, task='multiclass'),
                                                     torchmetrics.F1Score(num_classes = self.n_classes,
                                                                     average = 'macro', task='multiclass'),
                                                     torchmetrics.Recall(average = 'macro',
                                                                         num_classes = self.n_classes, task='multiclass'),
                                                     torchmetrics.Precision(average = 'macro',
                                                                            num_classes = self.n_classes, task='multiclass'),
                                                     torchmetrics.Specificity(average = 'macro',
                                                                            num_classes = self.n_classes, task='multiclass')])
        else : 
            self.AUROC = torchmetrics.AUROC(num_classes=2, average = 'macro', task='multiclass')
            metrics = torchmetrics.MetricCollection([torchmetrics.Accuracy(num_classes = 2,
                                                                           average = 'micro', task='multiclass'),
                                                     torchmetrics.CohenKappa(num_classes = 2, task='multiclass'),
                                                     torchmetrics.F1Score(num_classes = 2,
                                                                     average = 'macro', task='multiclass'),
                                                     torchmetrics.Recall(average = 'macro',
                                                                         num_classes = 2, task='multiclass'),
                                                     torchmetrics.Precision(average = 'macro',
                                                                            num_classes = 2, task='multiclass')])
        self.balance_accuracy_score = sklearn.metrics.balanced_accuracy_score
        self.valid_metrics = metrics.clone(prefix = 'val_')
        self.test_metrics = metrics.clone(prefix = 'test_')

        #--->random
        self.shuffle = CONFIG['data_shuffle']
        self.count = 0

        self.validation_step_outputs = []
        self.test_step_outputs = []

        
    def forward(self, feats):
        results_dict = self.model(data=feats)
        return results_dict
    
        
        
    def configure_optimizers(self):
        optimizer = optim.RAdam(self.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.CyclicLR(
            optimizer, base_lr=CONFIG['lr']*0.5, max_lr=CONFIG['lr'] * 1.5,
            step_size_up=5, cycle_momentum=False, mode="triangular2", verbose=True
        )
        return [optimizer], [scheduler]
    

    #---->remove v_num
    def get_progress_bar_dict(self):
        # don't show the version number
        items = super().get_progress_bar_dict()
        items.pop("v_num", None)
        return items

    
    def training_step(self, batch, batch_idx):
        """每一步的训练/一个epoch中的 training_step"""
        #---->inference
        data, label = batch
        results_dict = self.forward(data) # 训练时模型返回三个参数
        logits = results_dict['logits']
        Y_prob = results_dict['Y_prob']
        Y_hat = results_dict['Y_hat']

        #---->loss
        loss = self.loss(logits, label)

        #---->acc log
        Y = int(label)
        self.data[Y]["count"] += 1
        self.data[Y]["correct"] += (Y_hat.item() == Y)
        
        return {'loss': loss} 

    
    def on_validation_epoch_start(self):
        """
            training_epoch 之后，开始 validation 之前进行的操作
        """
        
        print(f"********************************** Epoch:{self.current_epoch} **********************************")
        print("Training:")
        for c in range(self.n_classes):
            count = self.data[c]["count"]
            correct = self.data[c]["correct"]
            if count == 0: 
                acc = None
            else:
                acc = float(correct) / count
            print('class {}({}): acc {}, correct {}/{}'.format(label_decoder[c], c, acc, correct, count))
        self.data = [{"count": 0, "correct": 0} for i in range(self.n_classes)]
        print()

        
    def validation_step(self, batch, batch_idx):
        """每一步的验证/一个epoch中的 valid_step"""
        data, label = batch
        results_dict = self(data) # 验证时模型返回三个参数
        logits = results_dict['logits']
        Y_prob = results_dict['Y_prob']
        Y_hat = results_dict['Y_hat']
        
        #---->acc log
        Y = int(label)
        self.data[Y]["count"] += 1
        self.data[Y]["correct"] += (Y_hat.item() == Y)
        
        self.validation_step_outputs.append({'logits' : logits, 'Y_prob' : Y_prob, 'Y_hat' : Y_hat, 'label' : label})
        return {'logits' : logits, 'Y_prob' : Y_prob, 'Y_hat' : Y_hat, 'label' : label}


    def on_validation_epoch_end(self):
        """
            vaild_step 之后进行的操作
        """
        logits = torch.cat([x['logits'] for x in self.validation_step_outputs], dim = 0) # [45, 5]
        probs = torch.cat([x['Y_prob'] for x in self.validation_step_outputs], dim = 0) # [45, 5]
        max_probs = torch.stack([x['Y_hat'] for x in self.validation_step_outputs])
        target = torch.stack([x['label'] for x in self.validation_step_outputs], dim = 0)

        #---->记录指标
        self.log('bal_acc', self.balance_accuracy_score(max_probs.cpu(), target.cpu()), prog_bar=True, on_epoch=True, logger=True)
        self.log('val_loss', cross_entropy_torch(logits, target), prog_bar=True, on_epoch=True, logger=True)
        self.log('auc', self.AUROC(probs, target.squeeze()), prog_bar=True, on_epoch=True, logger=True)
        self.log_dict(self.valid_metrics(max_probs.squeeze(0) , target.squeeze(0)),
                          on_epoch = True, logger = True)

        #---->acc log
        print("Validation:")
        for c in range(self.n_classes):
            count = self.data[c]["count"]
            correct = self.data[c]["correct"]
            if count == 0: 
                acc = None
            else:
                acc = float(correct) / count
            print('class {}({}): acc {}, correct {}/{}'.format(label_decoder[c], c, acc, correct, count))
        self.data = [{"count": 0, "correct": 0} for i in range(self.n_classes)]
        
        #---->random, if shuffle data, change seed
        if self.shuffle == True:
            self.count = self.count+1
            random.seed(self.count*50)
            
        print()
        self.validation_step_outputs.clear()
    
    
    def test_step(self, batch, batch_idx):
        """测试时才会进行的 test_step"""
        data, label = batch
        results_dict = self(data)
        logits = results_dict['logits']
        Y_prob = results_dict['Y_prob']
        Y_hat = results_dict['Y_hat']

        #---->acc log
        Y = int(label)
        self.data[Y]["count"] += 1
        self.data[Y]["correct"] += (Y_hat.item() == Y)
        
        self.test_step_outputs.append({'logits' : logits, 'Y_prob' : Y_prob, 'Y_hat' : Y_hat, 'label' : label})
        return {'logits' : logits, 'Y_prob' : Y_prob, 'Y_hat' : Y_hat, 'label' : label}

    
    def on_test_epoch_end(self):
        """
            test_step 之后会进行的操作
        """
        probs = torch.cat([x['Y_prob'] for x in self.test_step_outputs], dim = 0)
        max_probs = torch.stack([x['Y_hat'] for x in self.test_step_outputs])
        target = torch.stack([x['label'] for x in self.test_step_outputs], dim = 0)
        
        #---->
        bal_acc = self.balance_accuracy_score(max_probs.cpu(), target.cpu())
        auc = self.AUROC(probs, target.squeeze())
        metrics = self.test_metrics(max_probs.squeeze() , target.squeeze())
        metrics['auc'] = auc
        metrics['bal_acc'] = bal_acc

        for keys, values in metrics.items():
            print(f'{keys} = {values}')
            metrics[keys] = values
                
        #---->acc log
        print("Test:")
        for c in range(self.n_classes):
            count = self.data[c]["count"]
            correct = self.data[c]["correct"]
            if count == 0: 
                acc = None
            else:
                acc = float(correct) / count
            print('class {}({}): acc {}, correct {}/{}'.format(label_decoder[c], c, acc, correct, count))
        self.data = [{"count": 0, "correct": 0} for i in range(self.n_classes)]
        #---->
        result = pd.DataFrame([metrics])
        os.makedirs(os.path.join(self.log_path, f"fold{CONFIG['fold']}"), exist_ok=True)
        result.to_csv(os.path.join(self.log_path, f"fold{CONFIG['fold']}", f"result_{CONFIG['model_path']}.csv"))
        print()
        self.test_step_outputs.clear()


    def load_model(self):
        try:
            Model = TransMIL
        except:
            raise ValueError('Invalid Module File Name or Invalid Class Name!')
        self.model = self.instancialize(Model)
        pass

    
    def instancialize(self, Model, **other_args):
        """ Instancialize a model using the corresponding parameters
            from self.hparams dictionary. You can also input any args
            to overwrite the corresponding value in self.hparams.
        """
        class_args = inspect.getargspec(Model.__init__).args[1:]
        inkeys = self.hparams.model.keys()
        args1 = {}
        for arg in class_args:
            if arg in inkeys:
                args1[arg] = self.hparams.model[arg]
        args1.update(other_args)
        return Model(**args1)

In [13]:
# Initialize seed
pl.seed_everything(CONFIG['seed'])

metrics_list = []
for fold in range(CONFIG['nfold']):
    CONFIG['fold'] = fold
    print()
    print(f"**************************************** Train fold{CONFIG['fold']} model ****************************************")
    
    # load loggers
    CONFIG['load_loggers'] = load_loggers()

    # load callbacks
    CONFIG['callbacks'] = load_callbacks()

    # Define Data
    DataInterface_dict = {
                            'train_batch_size': CONFIG['train_dataloader']['batch_size'],
                            'train_num_workers': CONFIG['train_dataloader']['num_workers'],
                            'test_batch_size': CONFIG['test_dataloader']['batch_size'],
                            'test_num_workers': CONFIG['test_dataloader']['num_workers'],
                            'dataset_name': CONFIG['dataset_name'],
                        }

    dm = DataInterface(**DataInterface_dict)
    dm.setup(stage='fit')
    # Define Model
    model_dict = {
        "name": CONFIG['name'],
        "n_classes": CONFIG['n_classes'],
    }
    model = ModelInterface(model_dict)

    trainer = Trainer(
            num_sanity_val_steps = 0,
            accelerator = CONFIG['device'],
            devices = CONFIG['devices'],
            logger = CONFIG['load_loggers'],
            callbacks = CONFIG['callbacks'],
            max_epochs = CONFIG['epochs'], 
            precision = CONFIG['precision'],  
            accumulate_grad_batches = CONFIG['grad_acc'],
#             fast_dev_run=4,
#             limit_train_batches=0.1,
#             limit_val_batches=0.1,
        )

    # 训练
    trainer.fit(model = model, datamodule = dm)
    
    metrics = pd.read_csv(f"{CONFIG['fold_log_path']}/metrics.csv")
    metrics = metrics.sort_values(by='val_loss', ascending=True)
    del metrics['step']
    metrics_list.append(metrics.set_index('epoch').head(CONFIG['save_top_k']))
    display(metrics_list[-1].style.format(precision=4).background_gradient())
    
    
    # 测试
    model_paths = list(CONFIG['fold_log_path'].glob('*.ckpt'))
    model_paths = [str(model_path) for model_path in model_paths if 'epoch' in str(model_path)]
    print(f"-------> fold{CONFIG['fold']} start test:")
    for idx, path in enumerate(model_paths):
        print(f"**************************************** Test model idx:{idx} ****************************************")
        print(path)
        CONFIG['model_path'] = os.path.splitext(os.path.basename(path))[0]
        new_model = ModelInterface(model_dict)
        ckpt = torch.load(path)
        new_model.load_state_dict(ckpt['state_dict'])
        trainer.test(model=new_model, datamodule=dm)
        


**************************************** Train fold0 model ****************************************
---->Log dir: logs/UBC/TransMIL/fold0
Adjusting learning rate of group 0 to 1.0000e-04.


Training: |          | 0/? [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.4000e-04.


Validation: |          | 0/? [00:00<?, ?it/s]

********************************** Epoch:0 **********************************
Training:
class MC(0): acc 0.18181818181818182, correct 6/33
class EC(1): acc 0.5268817204301075, correct 49/93
class CC(2): acc 0.5733333333333334, correct 43/75
class HGSC(3): acc 0.7365269461077845, correct 123/167
class LGSC(4): acc 0.03333333333333333, correct 1/30

Validation:
class MC(0): acc 0.125, correct 1/8
class EC(1): acc 0.9166666666666666, correct 22/24
class CC(2): acc 0.8421052631578947, correct 16/19
class HGSC(3): acc 0.7142857142857143, correct 30/42
class LGSC(4): acc 0.0, correct 0/7

Adjusting learning rate of group 0 to 1.8000e-04.


...

********************************** Epoch:22 **********************************
Training:
class MC(0): acc 1.0, correct 33/33
class EC(1): acc 1.0, correct 93/93
class CC(2): acc 1.0, correct 75/75
class HGSC(3): acc 1.0, correct 167/167
class LGSC(4): acc 1.0, correct 30/30

Validation:
class MC(0): acc 0.875, correct 7/8
class EC(1): acc 0.75, correct 18/24
class CC(2): acc 0.8947368421052632, correct 17/19
class HGSC(3): acc 0.9761904761904762, correct 41/42
class LGSC(4): acc 0.8571428571428571, correct 6/7



,val_MulticlassF1Score,auc,val_MulticlassCohenKappa,val_MulticlassPrecision,val_loss,val_MulticlassRecall,val_MulticlassSpecificity,bal_acc,val_MulticlassAccuracy
epoch,,,,,,,,,
12,0.9195,0.9935,0.8741,0.9278,0.2201,0.9155,0.9727,0.9278,0.9100


-------> fold3 start test:
**************************************** Test model idx:0 ****************************************
logs/UBC/TransMIL/fold3/epoch=12-val_loss=0.2201.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

test_MulticlassAccuracy = 0.36000001430511475
test_MulticlassCohenKappa = 0.19999998807907104
test_MulticlassF1Score = 0.29666668176651
test_MulticlassRecall = 0.36000001430511475
test_MulticlassPrecision = 0.3104761838912964
test_MulticlassSpecificity = 0.8400000333786011
auc = 0.8880000114440918
bal_acc = 0.5174603174603174
Test:
class MC(0): acc 0.0, correct 0/5
class EC(1): acc 0.4, correct 2/5
class CC(2): acc 0.6, correct 3/5
class HGSC(3): acc 0.8, correct 4/5
class LGSC(4): acc 0.0, correct 0/5



In [14]:
metrics_list = [metric.reset_index(drop=True) for metric in metrics_list]

for metric in metrics_list:
    display(metric.head(CONFIG['save_top_k']).style.format(precision=4).background_gradient())

final_metric = pd.concat(metrics_list, axis=0)
final_metric.mean(0)

,val_MulticlassF1Score,auc,val_MulticlassCohenKappa,val_MulticlassPrecision,val_loss,val_MulticlassRecall,val_MulticlassSpecificity,bal_acc,val_MulticlassAccuracy
0,0.8721,0.9837,0.8181,0.9044,0.3400,0.8502,0.9630,0.9044,0.8700


,val_MulticlassF1Score,auc,val_MulticlassCohenKappa,val_MulticlassPrecision,val_loss,val_MulticlassRecall,val_MulticlassSpecificity,bal_acc,val_MulticlassAccuracy
0,0.9048,0.9840,0.8165,0.9172,0.3014,0.9024,0.9586,0.9172,0.8700


,val_MulticlassF1Score,auc,val_MulticlassCohenKappa,val_MulticlassPrecision,val_loss,val_MulticlassRecall,val_MulticlassSpecificity,bal_acc,val_MulticlassAccuracy
0,0.9859,0.9979,0.9723,0.9820,0.0831,0.9905,0.9949,0.9820,0.9800


,val_MulticlassF1Score,auc,val_MulticlassCohenKappa,val_MulticlassPrecision,val_loss,val_MulticlassRecall,val_MulticlassSpecificity,bal_acc,val_MulticlassAccuracy
0,0.9195,0.9935,0.8741,0.9278,0.2201,0.9155,0.9727,0.9278,0.9100


val_MulticlassF1Score        0.920566
auc                          0.989802
val_MulticlassCohenKappa     0.870250
val_MulticlassPrecision      0.932841
val_loss                     0.236121
val_MulticlassRecall         0.914630
val_MulticlassSpecificity    0.972300
bal_acc                      0.932841
val_MulticlassAccuracy       0.907500
dtype: float64